In [14]:
import EddyDetectionV2 as eddy
import oceanspy as ospy
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean.cm as cmo
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.colors import Normalize
from matplotlib.collections import LineCollection
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.colors import SymLogNorm
from tqdm import tqdm
import ast
from matplotlib.animation import FuncAnimation, FFMpegFileWriter
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd

# Set default font size to match LaTeX document
plt.rcParams.update({
    'font.size': 11,       # Default text size
    'axes.titlesize': 10,  # Title size
    'axes.labelsize': 10,  # Axis label size
    'xtick.labelsize': 8, # X-tick label size
    'ytick.labelsize': 8, # Y-tick label size
    'legend.fontsize': 8, # Legend font size
    'figure.titlesize': 12 # Figure title size
})

In [11]:
from pathlib import Path

def get_file_size(file_path):
    file_size_bytes = Path(file_path).stat().st_size
    if file_size_bytes < 1024:
        return f"{file_size_bytes} B"
    elif file_size_bytes < 1024**2:
        return f"{file_size_bytes / 1024:.2f} KB"
    elif file_size_bytes < 1024**3:
        return f"{file_size_bytes / 1024**2:.2f} MB"
    else:
        return f"{file_size_bytes / 1024**3:.2f} GB"

file_path = '/nird/projects/NS9608K/MSc_EK/Data/EddyResults/Tracking/EddyAreaID.nc'
print(f"File size: {get_file_size(file_path)}")

import netCDF4 as nc

# Open NetCDF file using netCDF4
ds_nc = nc.Dataset('/nird/projects/NS9608K/MSc_EK/Data/EddyResults/Tracking/EddyAreaID.nc')

# Extract variable data excluding coordinates
data_vars = {}
for var_name, nc_var in ds_nc.variables.items():
    if var_name not in ds_nc.dimensions:
        data_vars[var_name] = (nc_var.dimensions, np.array(nc_var))

# Extract coordinate data
coords = {}
for coord_name, nc_coord in ds_nc.dimensions.items():
    coords[coord_name] = np.array(ds_nc.variables[coord_name])

# Extract global attributes
attrs = {attr: ds_nc.getncattr(attr) for attr in ds_nc.ncattrs()}

# Create xarray Dataset with extracted information
ds_xr = xr.Dataset(data_vars=data_vars, coords=coords, attrs=attrs)

# Display the converted xarray.Dataset
print(ds_xr)

JMREI = ds_xr['EddyID'].sel(Y=slice(71, 74), X=slice(-18, -7))
display(JMREI)

File size: 2.26 GB
<xarray.Dataset> Size: 346MB
Dimensions:  (Y: 231, X: 510, time: 365)
Coordinates:
  * X        (X) float64 4kB -21.98 -21.93 -21.87 -21.82 ... 1.879 1.936 1.994
  * Y        (Y) float64 2kB 70.01 70.03 70.04 70.06 ... 74.89 74.92 74.95 74.99
  * time     (time) int64 3kB 0 1 2 3 4 5 6 7 ... 358 359 360 361 362 363 364
Data variables:
    XC       (Y, X) float64 942kB -21.98 -21.93 -21.87 ... 1.879 1.936 1.994
    YC       (Y, X) float64 942kB 70.01 70.01 70.01 70.01 ... 74.99 74.99 74.99
    Z        float64 8B -1.0
    EddyID   (time, Y, X) object 344MB '0' '0' '0' '0' '0' ... '0' '0' '0' '0'


<xarray.DataArray 'EddyID' (time: 365, Y: 139, X: 248)> Size: 101MB
array([[['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ...,
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0']],

       [['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ...,
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0']],

       [['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ...,
...
        ...,
        ['0', '0', '0', ..., "['3084']", '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0']],

       [['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ...,
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0']],

       [['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ...,
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0'],
        ['0', '0', '0', ..., '0', '0', '0']]], dtype=object)
Coordinates:
  * X        (X) float64 2kB -17.99 -17.94 -17.9 -17.85 ... -7.129 -7.085 -7.041
  * Y        (Y) float64 1kB 71.01 71.03 71.04 71.06 ... 73.9 73.93 73.95 73.98
  * time     (time) int64 3kB 0 1 2 3 4 5 6 7 ... 358 359 360 361 362 363 364

In [12]:
JMFZ_region = xr.open_dataset(r'/nird/projects/NS9608K/MSc_EK/Data/FWT/JMFZFWT.nc')['shelfExtended'].assign_attrs(
    units="mSv", description="Freshwater transport"
).rename('FWT_70_74N')
eddyRegion = xr.open_dataset('/nird/projects/NS9608K/MSc_EK/Data/Eddies_fullYear.nc')['EddyDetection'].sel(Y=slice(71,74),X=slice(-18,-7))
eddyRegion.attrs = {}
eddyRegion = eddyRegion.assign_attrs(
    unit="Eddy type (1 == anti-cyclonic, 2 == cyclonic)", description="Outermost closed SSH defined eddy region"
)

depth = xr.open_dataset(r'/nird/projects/NS9608K/MSc_EK/Data/Depth_res.nc')['Depth'].sel(Y=slice(71,74),X=slice(-18,-7))
depth_no_nan = depth

depth = depth.where(depth > 0, np.nan)

FWC = xr.open_dataset(r'/nird/projects/NS9608K/MSc_EK/Data/FWC_full_domain.nc')['__xarray_dataarray_variable__'].sel(Y=slice(71,74)).sel(X=slice(-18,-7))
FWC = FWC.resample(time='D').mean(dim='time').rename('FWC')

area = xr.open_dataset('/nird/projects/NS9608K/MSc_EK/Data/rA.nc')['rA'].sel(Y=slice(71,74),X=slice(-18,-7))

FWT_restructure = xr.open_dataset('/nird/projects/NS9608K/MSc_EK/Data/FWT/JMFZFWT_regrid.nc')['Jan Mayen region FWT']

In [19]:
locMax_surf = pd.read_csv('/nird/projects/NS9608K/MSc_EK/Data/EddyResults/Tracking/locMAX.csv')
locMin_surf = pd.read_csv('/nird/projects/NS9608K/MSc_EK/Data/EddyResults/Tracking/locMin.csv')

eddies_ID_max_t = locMax_surf.set_index(['Time'])
eddies_ID_min_t = locMin_surf.set_index(['Time'])

In [20]:
display(eddies_ID_max_t)

,ID,Latitude,Longitude
Time,,,
0,['1'],71.079446,-9.051356
0,['2'],71.241831,-14.804034
0,['3'],71.278327,-10.910303
0,['4'],71.370250,-2.622462
0,['5'],71.444517,-8.921229
...,...,...,...
364,['3140'],72.949224,-6.374665
364,['3170'],73.281687,-17.847658
364,['3156'],73.330735,-15.751358


In [33]:
time_ = np.arange(0,365)
for time in time_:
    fig, ax = plt.subplots(
        figsize=(6.2217,5), subplot_kw=dict(projection=ccrs.NorthPolarStereo(central_longitude=-10)), layout = 'constrained'
    )

    # Set the extent to focus on the desired geographical area
    extent = [-18+10, -7+10, 71, 74]

    lon = depth['XC']
    lat = depth['YC']

    land = depth_no_nan.where(depth_no_nan==0)

    mesh = ax.pcolormesh(FWC.X,FWC.Y,FWC[time],cmap='binary',vmin=0,vmax=5, transform=ccrs.PlateCarree(),zorder=0)
    cbar = fig.colorbar(mesh, shrink=0.7,pad=0.01,extend='max')
    cbar.set_label('FWC [m]')

    ax.pcolormesh(land.X,land.Y,land,cmap='Greys',vmin=-25,vmax=100,zorder=0, transform=ccrs.PlateCarree())

    # Add land contour
    ax.contour(lon, lat, depth_no_nan, [0], colors='black', linewidths=0.25,zorder=0, transform=ccrs.PlateCarree())
    ax.contour(lon, lat, depth_no_nan, [300,400,500,750,1000,1500,2000], colors='grey', linewidths=0.25,alpha=0.3, transform=ccrs.PlateCarree())

    # Eastern boundary
    ax.plot(JMFZ_region.XC,JMFZ_region.YC,linestyle='--',color='y',linewidth=0.75,label='Eastern boundary',transform=ccrs.PlateCarree())

    # Ridge
    ax.plot([-16,-15,-14,-13,-12,-11],[72.5,72.5,72.5,72.5,72.5,72.4],linestyle='--',color='c',label='Ridge',transform=ccrs.PlateCarree())

    # Eddies
    ax.contourf(eddyRegion.X.sel(X=slice(-18,-7)),eddyRegion.Y.sel(Y=slice(71,74)),eddyRegion[time].sel(X=slice(-18,-7),Y=slice(71,74)),[0.5,1.5,3.5,4.5],colors=['red','blue','green'],alpha=0.2,transform=ccrs.PlateCarree())
    

    # ID
    ID_max_lon = eddies_ID_max_t.at[time,'Longitude'].values
    ID_max_lat = eddies_ID_max_t.at[time,'Latitude'].values
    ID_max = eddies_ID_max_t.at[time,'ID'].values

    #display(ID_max,ID_max_lon,ID_max_lat)

    for data in range(len(ID_max)):
        if -18 <= ID_max_lon[data] <= -7 and 71 <= ID_max_lat[data] <= 74:
            if len(str(ID_max[data])) < 30:
                ax.annotate(ID_max[data],xy=[ID_max_lon[data],ID_max_lat[data]],fontsize=4,
                            bbox=dict(boxstyle='round,pad=0.01', facecolor='white', alpha=0.3),
                            transform=ccrs.PlateCarree())
            else:
                ax.annotate(str(ID_max[data])[0:30]+'...',xy=[ID_max_lon[data],ID_max_lat[data]],fontsize=4,
                            bbox=dict(boxstyle='round,pad=0.01', facecolor='white', alpha=0.3),
                            transform=ccrs.PlateCarree())

    ID_min_lon = eddies_ID_min_t.at[time,'Longitude'].values
    ID_min_lat = eddies_ID_min_t.at[time,'Latitude'].values
    ID_min = eddies_ID_min_t.at[time,'ID'].values

    for data in range(len(ID_min)):
        if -18 <= ID_min_lon[data] <= -7 and 71 <= ID_min_lat[data] <= 74:
            if len(str(ID_min[data])) < 30:
                ax.annotate(ID_min[data],xy=[ID_min_lon[data],ID_min_lat[data]],fontsize=4,
                            bbox=dict(boxstyle='round,pad=0.01', facecolor='white', alpha=0.3),
                            transform=ccrs.PlateCarree())
            else:
                ax.annotate(str(ID_min[data])[0:30] + '...',xy=[ID_min_lon[data],ID_min_lat[data]],fontsize=4,
                            bbox=dict(boxstyle='round,pad=0.01', facecolor='white', alpha=0.3),
                            transform=ccrs.PlateCarree())


    # Title
    ax.set_title(str(eddyRegion.time[time].values)[0:10])

    # Create a custom path for the extent in PlateCarree projection
    num_points = 100
    bottom_lons = np.linspace(extent[0], extent[1], num_points)
    top_lons = np.linspace(extent[1], extent[0], num_points)
    bottom_lats = np.full_like(bottom_lons, extent[2])
    top_lats = np.full_like(top_lons, extent[3])

    # Combine the vertices
    verts = np.vstack([
        np.column_stack([bottom_lons, bottom_lats]),
        np.column_stack([top_lons, top_lats])
    ])

    # Transform the vertices to the NorthPolarStereo projection
    proj = ccrs.NorthPolarStereo()
    verts_proj = proj.transform_points(ccrs.PlateCarree(), verts[:, 0], verts[:, 1])
    verts_proj = verts_proj[:, :2]  # Only keep x and y coordinates

    # Create the path
    codes = [Path.MOVETO] + [Path.LINETO] * (len(verts_proj) - 1) + [Path.CLOSEPOLY]
    path = Path(np.vstack([verts_proj, verts_proj[0]]), codes)
    patch = PathPatch(path, transform=ax.transData, edgecolor='black', facecolor='none')
    ax.add_patch(patch)

    # Set the boundary using the transformed path
    ax.set_boundary(patch.get_path(), transform=ax.transData)

    # Add gridlines without labels
    gl = ax.gridlines(draw_labels=False,alpha=0.5)

    # Manually add the labels for the bottom and right sides
    xticks = np.arange(extent[0]-9, extent[1]-10 + 1, 3)  # Adjust the range and step as needed
    yticks = np.arange(extent[2], extent[3] + 0.5, 1)  # Adjust the range and step as needed

    # Add bottom labels
    for xtick in xticks:
        ax.text(xtick, extent[2]-0.2, f'{xtick}°E', transform=ccrs.PlateCarree(),
                fontsize=10, ha='center', va='top')

    # Add right labels
    for ytick in yticks:
        ax.text(extent[1] + 1-10, ytick, f'{ytick}°N', transform=ccrs.PlateCarree(),
                fontsize=10, ha='left', va='center')
    
    ax.legend(loc='upper right')


    # Save the figure
    fig.savefig(f'/nird/home/ekv036/MSc/Fig/EddyTracking/JMR_tracking/{str(eddyRegion.time[time].values)[0:10]}',dpi=300, facecolor='w', edgecolor='w',
            orientation='landscape', format=None,
            transparent=False, bbox_inches=None,pad_inches=0.25)
    plt.close()

In [34]:
from PIL import Image
import os

# Define the folder containing the PNG files and the output GIF file
input_folder = '/nird/home/ekv036/MSc/Fig/EddyTracking/JMR_tracking'
output_gif = '/nird/home/ekv036/MSc/Animations/JMFZ_region_365.gif'

# Get a list of all PNG files in the folder
png_files = [f for f in os.listdir(input_folder) if f.endswith('.png')]

# Sort the files if necessary (e.g., if they need to be in a specific order)
png_files.sort()

# Open the images and store them in a list
images = [Image.open(os.path.join(input_folder, file)) for file in png_files]

# Save the images as a GIF
images[0].save(output_gif, save_all=True, append_images=images[1:], optimize=False, duration=150, loop=0)

print(f"GIF saved as {output_gif}")

GIF saved as /nird/home/ekv036/MSc/Animations/JMFZ_region_365.gif
